In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url = 'https://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=10&vw=div&SecurityToken=0P000003MC%5D3%5D0%5DE0WWE%24%24ALL&Id=0P000003MC&ClientFund=0&CurrencyId=BAS'


In [3]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)


In [4]:
#Store contents of the website under doc
doc = lh.fromstring(page.content)


In [5]:
#Parse data stored between <tr>
tr_elements = doc.xpath('//tr')
tr_elements.pop(0)

<Element tr at 0x244e64f8818>

In [6]:
#Check length of first 12 rows
[len(T) for T in tr_elements[:20]]

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4]

In [7]:
#Parse first row as header (financial years)
#Create empty list
col=[]
i=0

#For each row, store each first elements (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Declared Date"
2:"Ex Date"
3:"Payment Date"
4:"Type"
5:"Currency"
6:"Amount"


In [8]:
#Since first row is header, data stored on second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #if row is not of size 6, the //tr data is not from our table (should just delete primary headings but retain line items)
    if len(T)!=6:
        continue
        
    #i is the index of our column
    i=0
    
    #iterate through each element of row
    for t in T.iterchildren():
        data = t.text_content()
        #check if row is empty
        if i>0:
            try:
                data=int(data)
            except:
                pass
        #Append data to empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        

In [9]:
#check column lengths
[len(C) for (title,C) in col]

[12, 12, 12, 12, 12, 12]

In [10]:
#Create DataFrame
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

df.head()

,Declared Date,Ex Date,Payment Date,Type,Currency,Amount
0,30/04/1996,07/05/1996,31/05/1996,Cash Dividend,USD,0.03
1,19/12/1995,26/12/1995,18/01/1996,Cash Dividend,USD,0.03
2,30/10/1995,08/11/1995,01/12/1995,Cash Dividend,USD,0.03
3,01/08/1995,09/08/1995,21/08/1995,Cash Dividend,USD,0.03
4,27/03/1995,30/05/1995,14/06/1995,Cash Dividend,USD,0.03
